In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn import svm

In [3]:
credit_df = pd.read_table('Credit.dat',
                         delimiter=' ',
                         usecols=np.arange(1,13))
credit_df.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Balance,Caucasian,Asian
0,14.891,3606.0,283.0,2.0,34.0,11.0,0.0,0.0,1.0,333.0,1.0,0.0
1,106.025,6645.0,483.0,3.0,82.0,15.0,1.0,1.0,1.0,903.0,0.0,1.0
2,104.593,7075.0,514.0,4.0,71.0,11.0,0.0,0.0,0.0,580.0,0.0,1.0
3,148.924,9504.0,681.0,3.0,36.0,11.0,1.0,0.0,0.0,964.0,0.0,1.0
4,55.882,4897.0,357.0,2.0,68.0,16.0,0.0,0.0,1.0,331.0,1.0,0.0


In [4]:
#  standardize all columns
credit_df_norm = (credit_df - credit_df.mean())/(credit_df.std())

In [5]:
credit_df_norm.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Balance,Caucasian,Asian
0,-0.860505,-0.489386,-0.464957,-0.698255,-1.256101,-0.783948,-1.034339,-0.332916,0.794400,-0.406768,1.003756,-0.584317
1,1.725276,0.827225,0.827667,0.030993,1.526539,0.495967,0.964384,2.996248,0.794400,0.833013,-0.993768,1.707122
2,1.684646,1.013518,1.028023,0.760241,0.888851,-0.783948,-1.034339,-0.332916,-1.255665,0.130471,-0.993768,1.707122
3,2.942467,2.065853,2.107363,0.030993,-1.140158,-0.783948,0.964384,-0.332916,-1.255665,0.965691,-0.993768,1.707122
4,0.302549,0.069925,0.013314,-0.698255,0.714936,0.815946,-1.034339,-0.332916,0.794400,-0.411118,1.003756,-0.584317


In [6]:
#  revert columns that range from 0 to 1
for col in credit_df.iteritems():
    if ((col[1].max() - col[1].min()) == 1.0):
        credit_df_norm[col[0]] = credit_df[col[0]]

In [7]:
credit_df_norm.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Balance,Caucasian,Asian
0,-0.860505,-0.489386,-0.464957,-0.698255,-1.256101,-0.783948,0.0,0.0,1.0,-0.406768,1.0,0.0
1,1.725276,0.827225,0.827667,0.030993,1.526539,0.495967,1.0,1.0,1.0,0.833013,0.0,1.0
2,1.684646,1.013518,1.028023,0.760241,0.888851,-0.783948,0.0,0.0,0.0,0.130471,0.0,1.0
3,2.942467,2.065853,2.107363,0.030993,-1.140158,-0.783948,1.0,0.0,0.0,0.965691,0.0,1.0
4,0.302549,0.069925,0.013314,-0.698255,0.714936,0.815946,0.0,0.0,1.0,-0.411118,1.0,0.0


In [115]:
target='Married'
training_points = 200
train_df = credit_df_norm.drop(target, axis=1).loc[:training_points]
train_target = credit_df_norm[target].replace(0, -1).loc[:training_points]

# train_df = np.array([[3,4],[1,4],[2,3],[6,-1],[7,-1],[5,-3],[2,4]] )
# train_target = np.array([-1,-1, -1, 1, 1 , 1, 1 ])

In [116]:
k_value = np.array(train_df @ train_df.T + np.identity(len(train_target))*1e-12)

In [117]:
k_value

array([[ 5.04153742, -4.94105406, -3.50945918, ...,  1.20452353,
         3.5809826 ,  0.51992661],
       [-4.94105406, 10.61708724,  6.69604535, ..., -2.7768132 ,
        -3.20264287, -0.79965593],
       [-3.50945918,  6.69604535,  7.92170028, ..., -3.7405506 ,
        -0.21268301, -2.95536785],
       ...,
       [ 1.20452353, -2.7768132 , -3.7405506 , ...,  6.70215302,
        -0.44118489, -0.08110758],
       [ 3.5809826 , -3.20264287, -0.21268301, ..., -0.44118489,
         6.93467274, -1.80715243],
       [ 0.51992661, -0.79965593, -2.95536785, ..., -0.08110758,
        -1.80715243,  4.35018343]])

In [118]:
#  check PSD
np.linalg.cholesky(k_value)

array([[ 2.24533682e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.20058480e+00,  2.40302180e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.56299899e+00,  1.35518268e+00,  1.90845863e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 5.36455609e-01, -6.64287416e-01, -1.04893038e+00, ...,
         1.02275371e-06,  0.00000000e+00,  0.00000000e+00],
       [ 1.59485320e+00,  1.27742011e-01,  1.10400988e+00, ...,
        -2.46664055e-08,  1.03358295e-06,  0.00000000e+00],
       [ 2.31558403e-01, -1.20719684e-01, -1.27319767e+00, ...,
         2.83861383e-10, -5.83309832e-09,  1.01751738e-06]])

In [119]:
alpha = cp.Variable(shape=train_target.shape)
beta = cp.multiply(alpha, train_target)
K = cp.Parameter(shape=k_value.shape, PSD=True, value=k_value)
obj = .5 * cp.quad_form(beta, K) - np.ones(alpha.shape).T @ alpha
const = [np.array(train_target.T) @ alpha == 0,
        -alpha <= np.zeros(alpha.shape),
        alpha <= 10*np.ones(shape=alpha.shape)]
prob = cp.Problem(cp.Minimize(obj), const)

In [120]:
result = prob.solve()
result

-1619.4756033909446

In [121]:
w = np.multiply(train_target, alpha.value).T @ train_df
w

Income      -0.053383
Limit       -0.509973
Rating       0.392604
Cards       -0.062476
Age         -0.188346
Education    0.022351
Gender       0.100693
Student     -0.698400
Balance      0.251143
Caucasian   -0.147832
Asian        0.095009
Name: Married, dtype: float64

In [147]:
S = (alpha.value > 1e-4).flatten()
b = train_target[S] - train_df[S] @ w
b = b[0]
b

0.874338454690015

In [124]:
def classify(x):
    result = w @ x + b
    return np.sign(result)

In [152]:
correct = 0
incorrect = 0
validation_set = credit_df_norm.drop(target, axis=1)
predictions = []
for i, x in validation_set.iterrows():
    my_svm = classify(x)
    if my_svm==credit_df_norm[target].replace(0, -1)[i]: correct +=1
    else: incorrect +=1
    predictions.append(my_svm)
predictions = np.array(predictions)

In [127]:
print(correct/(correct + incorrect))

0.61


In [175]:
np.array([i for i in alpha.value if i >=1e-4])

array([ 4.02718492, 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        ,  4.1578321 , 10.        ,  9.93494512,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.  

In [157]:
sklearn_svm.predict(credit_df_norm.drop(target, axis=1)) - predictions

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [177]:
sklearn_svm = svm.SVC(C = 10, kernel='linear')
sklearn_svm.fit(train_df, train_target)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [178]:
sklearn_svm.score(credit_df_norm.drop(target, axis=1), credit_df_norm[target])

0.6075

In [179]:
sklearn_svm.intercept_

array([0.87383422])

In [180]:
sklearn_svm.dual_coef_

array([[-10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.        , -10.        ,
        -10.        , -10.        , -10.